In [22]:
import chess.pgn
import time
import sys
import math
import numpy as np

In [35]:
BOARD_SPACES = 64
ROWS = 8
COLS = 8

letter_lookup = {
    'a':0,
    'b':1,
    'c':2,
    'd':3,
    'e':4,
    'f':5,
    'g':6,
    'h':7
}
piece_lookup = {
    'r':1,
    'n':2,
    'b':3,
    'q':4,
    'k':5,
    'p':6,
    'R':7,
    'N':8,
    'B':9,
    'Q':10,
    'K':11,
    'P':12,
    '.':0
}

In [53]:
# Format of Dataset
# [
#   [0 0 0 0 0 0]
#   [0 0 0 0 0 0]
#        ...
#   [0 0 0 0 0 0],
#   12, 18

wdataset_x = []
wdataset_y = []
bdataset_x = []
bdataset_y = []

posns = []

pgn = open("datasets/Adams.pgn")
game = chess.pgn.read_game(pgn)
board = game.board()
black_or_white = True
count = 0

while game != None:
#     print(count)
    count += 1
#     if count >= 100:
#         break

    for move in game.mainline_moves():
        dp = []
        
        # Create a one-hot label for the move
        label = np.zeros(BOARD_SPACES*2)
        pos1 = str(move)[:2]
        pos1 = ROWS*letter_lookup[pos1[0]] + int(pos1[1])-1
        label[pos1] = 1
        pos2 = str(move)[2:]
        pos2 = ROWS*letter_lookup[pos2[0]] + int(pos2[1])-1
        label[(BOARD_SPACES) + pos2] = 1
        
        posns.append((pos1, pos2))
        
        # Turn the board into a vector
        strboard = str(board)
        board_data = []
        for char in strboard:
            if char in piece_lookup:
                board_data.append(piece_lookup[char])
        
        if black_or_white:
            wdataset_x.append(board_data)
            wdataset_y.append(label)
        else:
            bdataset_x.append(board_data)
            bdataset_y.append(label)
            
        black_or_white = not black_or_white
        board.push(move)
    game = chess.pgn.read_game(pgn)

print(wdataset_x[0])
print(wdataset_y[0])

[1, 2, 3, 4, 5, 3, 2, 1, 6, 6, 6, 6, 6, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 12, 12, 12, 12, 12, 12, 7, 8, 9, 10, 11, 9, 8, 7]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]


In [75]:
wdataset_x = np.array(wdataset_x)
wdataset_x = wdataset_x.reshape(len(wdataset_x), 64)
wdataset_y = np.array(wdataset_y)

inshape = wdataset_x[0].shape
outshape = wdataset_y[0].size
print(wdataset_x.shape, outshape)

(129911, 64) 128


In [76]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD

In [80]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=64))
model.add(Dense(128, activation='relu', input_dim=64))
model.add(Dense(outshape, activation='softmax'))
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(wdataset_x, wdataset_y, epochs=100, batch_size=32)

Epoch 1/100
129911/129911 [==============================] - 6s 42us/step - loss: 9.4803 - acc: 8.6983e-04
Epoch 2/100
129911/129911 [==============================] - 5s 42us/step - loss: 9.4799 - acc: 9.4680e-04
Epoch 3/100
129911/129911 [==============================] - 6s 43us/step - loss: 9.4795 - acc: 9.3141e-04
Epoch 4/100
129911/129911 [==============================] - 6s 43us/step - loss: 9.4793 - acc: 0.0010
Epoch 5/100
129911/129911 [==============================] - 6s 43us/step - loss: 9.4794 - acc: 9.3141e-04
Epoch 6/100
129911/129911 [==============================] - 6s 46us/step - loss: 9.4789 - acc: 0.0011
Epoch 7/100
129911/129911 [==============================] - 6s 44us/step - loss: 9.4785 - acc: 0.0011
Epoch 8/100
129911/129911 [==============================] - 5s 42us/step - loss: 9.4782 - acc: 0.0011
Epoch 9/100
129911/129911 [==============================] - 6s 47us/step - loss: 9.4780 - acc: 0.0011
Epoch 10/100
129911/129911 [=============================

In [9]:
score = model.evaluate(wdataset_x, wdataset_y, batch_size=128)
print(score)

129911/129911 [==============================] - 2s 16us/step
[27.934931087176032, 0.4722309889153611]


In [12]:
print(model.predict(wdataset_x)[0])
print(wdataset_y[0])

[0.5075639 0.4924361]
[10 20]


In [ ]:
model = Sequential()
# Dense(64) is a fully-connected layer with 64 hidden units.
# in the first layer, you must specify the expected input data shape:
# here, 20-dimensional vectors.
model.add(Dense(64, activation='relu', input_dim=64))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model.fit(wdataset_x, wdataset_y,
          epochs=20,
          batch_size=128)
score = model.evaluate(wdataset_x, wdataset_y, batch_size=128)

In [ ]:
score